In [ ]:
!git clone https://github.com/M-Jafarkhani/Multimodal-Emotion-Recognition

In [ ]:
import gdown

file_id = "1_XdzdW8UNG1TTS6QcX10uhoS6N11OBit"
destination = "mosi_data.pkl"

gdown.download(
    f"https://drive.google.com/uc?id={file_id}", destination, quiet=False)

In [ ]:
import sys
import torch
import matplotlib.pyplot as plt

sys.path.append('/content/Multimodal-Emotion-Recognition/src')

In [ ]:
from loader import get_dataloader
from unimodals.modules import Transformer, MLP, Sequential, Identity
from training.supervised import train, test
from fusions.mulg import MulGFusion
from utils import get_default_device, save_results

In [ ]:
FILE_PATH = '/content/mosi_data.pkl'
train_data, valid_data, test_data = get_dataloader(FILE_PATH)

In [ ]:
device = get_default_device()
print(device)

In [ ]:
encoders = [Identity().to(device), Identity().to(device), Identity().to(device)]
head = nn.Sequential(
    nn.Linear(3 * 40, 40).to(device),
    nn.ReLU().to(device),
    nn.Linear(40, 2).to(device)  # Acc2
)
fusion = MulGFusion(d_l=50, d_a=50, d_v=50, hidden_dim=40).to(device)


In [ ]:
OUTPUT_MODEL_FILE_NAME = "MulG_Model"

train_losses, valid_losses = train(
    encoders=encoders,
    fusion=fusion,
    head=head,
    train_dataloader=train_loader,
    valid_dataloader=valid_loader,
    total_epochs=100,
    is_packed=False,      # padding already handled
    task="regression",
    optimtype=torch.optim.AdamW,
    early_stop=True,
    lr=1e-4,
    save=f"/content/{OUTPUT_MODEL_FILE_NAME}.pt",
    weight_decay=0.01,
    objective=torch.nn.L1Loss()
)


In [ ]:
plt.plot(train_losses, label='Train Loss')
plt.plot(valid_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.title('Loss vs. No. of epochs')

In [ ]:
save_results(train_losses, valid_losses, f'/content/{OUTPUT_MODEL_FILE_NAME}.pkl')

In [ ]:
model = torch.load(f"/content/{OUTPUT_MODEL_FILE_NAME}.pt").to(device)

test(
    model,
    test_data,
    is_packed=True,
    criterion=torch.nn.L1Loss(),
    task="posneg-classification",
)